In [ ]:
pip install tensorflow

In [ ]:
pip install nltk

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
from nltk.corpus import stopwords
import os
import nltk
import re
from nltk.stem import ISRIStemmer
nltk.download('punkt')
nltk.download('stopwords')
from tensorflow.keras.preprocessing.text import Tokenizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
test = pd.read_csv("/kaggle/input/dataset/test _no_label.csv")
train = pd.read_excel("/kaggle/input/dataset/train.xlsx")


train.columns = ['text','sentiment']
train_text = train['text']

# print(train)
# print(test)

In [ ]:
# lemm
!pip install qalsadi

In [ ]:
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
import qalsadi.lemmatizer

# Create an instance of the Lemmatizer
lemer = qalsadi.lemmatizer.Lemmatizer()

def solve(word_list):
    arabic_reg = re.compile('[\u0600-\u06FF]+')
    arabic_words = [word for word in word_list if arabic_reg.fullmatch(word)]
    lemmatized_words = [lemer.lemmatize_text(word) for word in arabic_words]
    return lemmatized_words

def remove_and_lemmatize(tokenized_corpus):
    filtered_corpus = []
    for tokenized_document in tokenized_corpus:
        filtered_document = solve(tokenized_document)
        filtered_corpus.append(filtered_document)
    return filtered_corpus

In [ ]:
# Tokenize the text
tokenized_train = [word_tokenize(sentence) for sentence in train_text]

# Remove stop words and lemmatize
stop_words_arabic = set(stopwords.words('arabic'))
filtered_text_train = [
    [word for word in sentence if word not in stop_words_arabic]
    for sentence in tokenized_train
]

resulttrain1 = remove_and_lemmatize(filtered_text_train)
# print(resulttrain1[0])
resulttrain=[]
for row in resulttrain1:
  x = []
  for lemma in row:
    for c in lemma:
      x.append(c)
  resulttrain.append(x)

# print(resulttrain)

result_train_df = pd.DataFrame({
    'review_description':resulttrain
})

# print(result_train_df)

cv = CountVectorizer()
todoc = [" ".join(doc) for doc in resulttrain]
x = cv.fit_transform(todoc)
feature_names = cv.get_feature_names_out()
xarr = x.toarray()
df = pd.DataFrame(xarr, columns=feature_names)

dftrain = pd.DataFrame({
    'review_description': resulttrain,
    'rating':train['sentiment']
})

# print(dftrain.head)

tokenized_test = [word_tokenize(sentence) for sentence in test['review_description']]

filtered_texttest = [
    [word for word in sentence if word not in stop_words_arabic]
    for sentence in tokenized_test
]

resulttest1=remove_and_lemmatize(filtered_texttest)

resulttest=[]
for row in resulttest1:
  x = []
  for lemma in row:
    for c in lemma:
      x.append(c)
  resulttest.append(x)

dftest = pd.DataFrame({
    'ID': test['ID'],
    'review_description': resulttest
})
# print(resulttest)

final=[]
for i in resulttest:
  sentcount=[]
  for j in feature_names:
    count = i.count(j)
    sentcount.append(count)
  final.append(sentcount)



dftestcount = pd.DataFrame({
    'review_description':final
})


In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Activation functions and their derivatives

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def initialize_parameters(input_size, hidden_size, output_size):
    Wxh = np.random.randn(input_size, hidden_size)
    Whh = np.random.randn(hidden_size, hidden_size)
    Why = np.random.randn(hidden_size, output_size)
    bh = np.zeros((1, hidden_size))
    by = np.zeros((1, output_size))
    return Wxh, Whh, Why, bh, by

def rnn_forward(inputs, h_prev, Wxh, Whh, Why, bh, by):
    h_t = sigmoid(np.dot(inputs, Wxh) + np.dot(h_prev, Whh) + bh)
    y_pred = sigmoid(np.dot(h_t, Why) + by)
    return h_t, y_pred

def rnn_backward(inputs, h_prev, h_t, y_pred, target, Wxh, Whh, Why, bh, by, learning_rate):
    dy = y_pred - target
    dWhy = np.dot(h_t.T, dy)
    dby = np.sum(dy, axis=0, keepdims=True)
    dh = np.dot(dy, Why.T) * h_t * (1 - h_t)
    dWhh = np.dot(h_prev.T, dh)
    dWxh = np.dot(inputs.T, dh)
    dbh = np.sum(dh, axis=0, keepdims=True)

    Wxh -= learning_rate * dWxh
    Whh -= learning_rate * dWhh
    Why -= learning_rate * dWhy
    bh -= learning_rate * dbh
    by -= learning_rate * dby

    return Wxh, Whh, Why, bh, by

def train_rnn(X, y, input_size, hidden_size, output_size, learning_rate, num_epochs):
    Wxh, Whh, Why, bh, by = initialize_parameters(input_size, hidden_size, output_size)

    for epoch in range(num_epochs):
        total_loss = 0.0
        h_prev = np.zeros((1, hidden_size))

        for inputs, target in zip(X, y):
            inputs = np.array([inputs])  # Convert inputs to a 2D array
            h_t, y_pred = rnn_forward(inputs, h_prev, Wxh, Whh, Why, bh, by)
            Wxh, Whh, Why, bh, by = rnn_backward(inputs, h_prev, h_t, y_pred, target, Wxh, Whh, Why, bh, by, learning_rate)

            if(y_pred<0.35):
              y_pred=-1
            elif(y_pred<0.65):
              y_pred=0
            else:
              y_pred=1
            #print(y_pred,target)
            loss = 0.5 * (y_pred - target)**2  # Mean squared error loss
            total_loss += loss

            h_prev = h_t

        average_loss = total_loss / len(y)
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}')

    return Wxh, Whh, Why, bh, by


def test_rnn(X_test, Wxh, Whh, Why, bh, by):
    cnt = 0
    h_prev = np.zeros((1, Wxh.shape[1]))
    predictions=[]
    for inputs in X_test:
        inputs = np.array([inputs])  # Convert inputs to a 2D array
        h_t, y_pred = rnn_forward(inputs, h_prev, Wxh, Whh, Why, bh, by)

        # Post-process y_pred based on your logic
        if y_pred < 0.35:
            y_pred = -1
        elif y_pred < 0.65:
            y_pred = 0
        else:
            y_pred = 1

        predictions.append(y_pred)
        h_prev = h_t

    return predictions

# Example usage:
X_train = np.array(df)
y_train = np.array(dftrain["rating"])
input_size = len(df.columns)
hidden_size = 20
output_size = 1
learning_rate = 0.01
num_epochs = 30

# Train the RNN and get the weights
trained_weights = train_rnn(X_train, y_train, input_size, hidden_size, output_size, learning_rate, num_epochs)

# Test the RNN using the trained weights
X_test = dftestcount['review_description']

predictions = test_rnn(X_test, *trained_weights)
final_df_test = pd.DataFrame({
    'ID': dftest['ID'],
    'review_description': dftest['review_description'],
    'rating_test': predictions
})
print(final_df_test)
final_df_test.to_csv('/kaggle/working/output.csv')